In [28]:
from abc import ABC
from pathlib import Path

import pandas as pd
from codetf.models import load_model_pipeline
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from sacrebleu import corpus_bleu, corpus_chrf, corpus_ter
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

In [2]:
root_dir = Path.cwd()

# Create Predictions
Im folgenden wird mit allen Models für den Testdatensatz die Predictions generiert.

In [10]:
class AbstractModel(ABC):
    def predict(self, code: str) -> str:
        raise NotImplementedError()
    
    def model_name(self) -> str:
        raise NotImplementedError()
    
class CodeTFModel(AbstractModel):
    def __init__(self, model_name: str, model_type: str, task: str) -> None:
        super().__init__()

        self._model = load_model_pipeline(model_name=model_name, model_type=model_type, task=task)
        self._model_name = model_name
        self._model_type = model_type
        self._task = task

    def predict(self, code: str) -> str:
        return self._model.predict([code])[0]
    
    def model_name(self) -> str:
        return f"{self._model_name}-{self._model_type}-{self._task}"
    
class SebisModel(AbstractModel):
    def __init__(self, model_name: str) -> None:
        super().__init__()

        self._pipeline = SummarizationPipeline(
            model=AutoModelWithLMHead.from_pretrained(model_name),
            tokenizer=AutoTokenizer.from_pretrained(model_name, skip_special_tokens=True),
            device=0
        )
        self._model_name = model_name

    def predict(self, code: str) -> str:
        return self._pipeline([code])[0]["summary_text"]
    
    def model_name(self) -> str:
        return self._model_name.replace("/", "-")

In [4]:
def get_preds(df: pd.DataFrame, model: AbstractModel):
    file_path = root_dir / "data" / "preds" / f"{model.model_name()}.csv"
    
    if file_path.exists():
        return

    df = df.copy()
    df["pred"] = df["code"].map(model.predict)
    df[["ref", "pred"]].to_csv(file_path)

In [5]:
dataset = load_dataset("json", data_files={
    "test": str(root_dir.parent / "data" / "test.jsonl"),
}, cache_dir=root_dir.parent / "data" / "cache")

Found cached dataset json (/home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-acdd91729f392843/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 19.11it/s]


In [6]:
def inference(doc):
    doc["ref"] = [" ".join(docstring) for docstring in doc["docstring_tokens"]]
    return doc    

dataset = dataset.map(inference, batched=True)
dataset.set_format(type="pandas", columns=["ref", "code"])
df = dataset["test"][:]
df.head()

Loading cached processed dataset at /home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-acdd91729f392843/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-e06db1d51f6ed421.arrow


,code,ref
0,def sina_xml_to_url_list(xml_data):\n rawur...,str - > list Convert XML to URL List . From Bi...
1,"def dailymotion_download(url, output_dir='.', ...",Downloads Dailymotion videos by URL .
2,"def sina_download(url, output_dir='.', merge=T...",Downloads Sina videos by URL .
3,"def sprint(text, *colors):\n return ""\33[{}...",Format text with color or other effects into A...
4,"def print_log(text, *colors):\n sys.stderr....",Print a log message to standard error .


In [7]:
get_preds(df, CodeTFModel(model_name="codet5", model_type="base-multi-sum", task="pretrained"))

In [8]:
get_preds(df, CodeTFModel(model_name="codet5", model_type="base", task="sum_python"))

In [11]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_source_code_summarization_python"))

In [12]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_source_code_summarization_python_transfer_learning_finetune"))

In [13]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_source_code_summarization_python_multitask"))

In [14]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_source_code_summarization_python_multitask_finetune"))

In [15]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_source_code_summarization_python"))

In [16]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune"))

In [17]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_source_code_summarization_python_multitask"))

In [18]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_source_code_summarization_python_multitask_finetune"))

In [19]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_large_source_code_summarization_python_transfer_learning_finetune"))

In [20]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask"))

In [21]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask_finetune"))

# Auswertung

In [34]:
def get_scores(df: pd.DataFrame, model_name: str):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    scores = [scorer.score(ref, hyp) for ref, hyp in zip(df["ref"].to_list(), df["pred"].to_list())]
    rouge1 = sum([score["rouge1"].fmeasure for score in scores]) / len(scores)
    rougeL = sum([score["rougeL"].fmeasure for score in scores]) / len(scores)
    # meteor = sum([meteor_score([ref], hyp) for ref, hyp in zip(df["ref"].to_list(), df["pred"].to_list())]) / len(df["pred"].to_list())

    return pd.DataFrame({
        "bleu": corpus_bleu(df["pred"].to_list(), [df["ref"].to_list()]).score,
        "chrf": corpus_chrf(df["pred"].to_list(), [df["ref"].to_list()]).score,
        "ter": corpus_ter(df["pred"].to_list(), [df["ref"].to_list()]).score,
        "rouge1": rouge1, "rougeL": rougeL
    }, index=pd.Index([model_name], name="Model"))

In [35]:
df = pd.DataFrame(columns=["bleu", "chrf", "ter"])

for csv_file in sorted((root_dir / "data" / "preds").glob("*.csv"), key=lambda f: f.name):
    df = pd.concat([df, get_scores(pd.read_csv(csv_file, index_col=0), csv_file.name)])

df

,bleu,chrf,ter,rouge1,rougeL
SEBIS-code_trans_t5_base_source_code_summarization_python.csv,0.483582,10.951082,167.123288,0.042248,0.031722
SEBIS-code_trans_t5_base_source_code_summarization_python_multitask.csv,2.308839,21.207518,98.630137,0.265339,0.265339
SEBIS-code_trans_t5_base_source_code_summarization_python_multitask_finetune.csv,1.650644,24.274855,135.616438,0.249998,0.206971
SEBIS-code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune.csv,1.964073,21.545830,109.589041,0.241978,0.214093
SEBIS-code_trans_t5_large_source_code_summarization_python_multitask.csv,7.233727,35.369698,104.109589,0.383260,0.361623
SEBIS-code_trans_t5_large_source_code_summarization_python_multitask_finetune.csv,1.587535,29.586688,120.547945,0.288733,0.243134
SEBIS-code_trans_t5_large_source_code_summarization_python_transfer_learning_finetune.csv,1.789650,25.225251,95.890411,0.288737,0.275404
SEBIS-code_trans_t5_small_source_code_summarization_python.csv,0.665183,11.690047,142.465753,0.025833,0.025833
SEBIS-code_trans_t5_small_source_code_summarization_python_multitask.csv,3.668414,22.595346,93.150685,0.371140,0.354473
SEBIS-code_trans_t5_small_source_code_summarization_python_multitask_finetune.csv,2.907154,20.997844,121.917808,0.267328,0.225341
